In [ ]:
from ols_split import *

results = pd.DataFrame(columns = ['Error', 'low_pc_high', 'low_effective_address_low', 'low_effective_address_high', 'high_pc_high', 'high_effective_address_low', 'high_effective_address_high'])
# files = ['1M-loads-split.csv', \
#          'compute_fp_1-loads-split.csv', 'compute_fp_4-loads-split.csv', 'compute_fp_9-loads-split.csv', 'compute_fp_10-loads-split.csv', \
#          'compute_int_0-loads-split.csv', 'compute_int_4-loads-split.csv', 'compute_int_7-loads-split.csv', 'compute_int_11-loads-split.csv', 'compute_int_15-loads-split.csv',\
#          'srv_10-loads-split.csv']

files = ['1M-loads-split.csv']

for file in files:
    dflo, dfhi, y0_range, y1_range = prepDataFrame(file)

    Xlo_tr, Xlo_te, ylo_tr, ylo_te = test_train_split(dflo, int(0.8*len(dflo)))

    # Predictor for the lower order bits
    sol_lo = solver(Xlo_tr, ylo_tr)
    temp = pd.DataFrame(Xlo_te, columns=['x1', 'x2', 'x3'])
    df   = pd.DataFrame()
    df['y0_prediction'] = (sol_lo[0]*temp["x1"] +  sol_lo[1]*temp["x2"] + sol_lo[2]*temp["x3"])*y0_range
    df['y0_truth'] = pd.Series(ylo_te)*y0_range

    # Predictor for the higher order bits
    Xhi_tr, Xhi_te, yhi_tr, yhi_te = test_train_split(dfhi, int(0.8*len(dfhi)))
    sol_hi = solver(Xhi_tr, yhi_tr)
    temp = pd.DataFrame(Xhi_te, columns=['x1', 'x2', 'x3'])
    df['y1_prediction'] = (sol_hi[0]*temp["x1"] +  sol_hi[1]*temp["x2"] + sol_hi[2]*temp["x3"])*y1_range
    df['y1_truth'] = pd.Series(yhi_te)*y1_range

    y_range = y1_range*2**32 + y0_range

    # Assemble the predictions from the high and low bits
    df['truth'] = (df['y1_truth']*2**32      + df['y0_truth'])/y_range
    df['pred']  = (df['y1_prediction']*2**32 + df['y0_prediction'])/y_range

    rootMSE = sqrt(((df['truth'] - df['pred'])**2).sum())/len(dflo)
    
    results.loc[file] = [rootMSE] + list(sol_lo) + list(sol_hi)

In [ ]:
results

In [ ]:
results.abs().to_csv('ols-split-RESULTS.csv')